# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

from pathlib import Path

from cleanup import DupInterface

# QGrid

In [ ]:
di = DupInterface.from_yaml(r'jsl\jsl.yaml')
display(di.widget)

In [ ]:
# change the table to show all the rows that are involved in duplicates
di.qg.df = di._df[di._df.duplicated(di.dup_bar.cols, keep=False)][di._cols]